In [1]:
import whisper    

# Initialize Whisper model
model = whisper.load_model("large-v2")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)>

In [4]:
import os
import shutil
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

from pydub import AudioSegment

class AudioFileHandler(FileSystemEventHandler):
    def __init__(self, model, output_dir, processed_dir):
        self.model = model
        self.output_dir = output_dir
        self.processed_dir = processed_dir

    def on_created(self, event):
        if not event.is_directory:
            self.process_file(event.src_path)

    def process_file(self, file_path):
        try:
            # Check if the file is an audio file
            if file_path.lower().endswith(('.wav', '.mp3', '.m4a')):
                print(f"Processing file: {file_path}")

                # Convert audio file to WAV if necessary
                if not file_path.lower().endswith('.wav'):
                    audio = AudioSegment.from_file(file_path)
                    output_audio_file = file_path[:-3] + "wav"
                    #audio.export(output_audio_file, format="wav")
                else:
                    output_audio_file = file_path

                # Transcribe audio using Whisper
                result = self.model.transcribe(output_audio_file)

                # Extract transcription text
                transcription_text = result['text']

                # Save transcription to a file
                transcript_file = os.path.join(self.output_dir, os.path.basename(file_path) + '.txt')
                with open(transcript_file, "w") as f:
                    f.write(transcription_text)

                print(f"Transcription saved to: {transcript_file}")

                # Move the processed audio file to another folder
                shutil.move(file_path, os.path.join(self.processed_dir, os.path.basename(file_path)))

                print(f"Moved {file_path} to {self.processed_dir}")

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

if __name__ == "__main__":
    watch_dir = "../transcriber/audio_files_drop"  # Directory to watch for new files
    output_dir = "../transcriber/transcripts"  # Directory to save transcripts
    processed_dir = "../transcriber/processed_audios"  # Directory to move processed audio files

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(processed_dir, exist_ok=True)

    # Initialize Whisper model
    model = whisper.load_model("large-v2")

    event_handler = AudioFileHandler(model, output_dir, processed_dir)
    observer = Observer()
    observer.schedule(event_handler, path=watch_dir, recursive=False)
    observer.start()

    print(f"Watching directory: {watch_dir}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

Watching directory: ../../transcriber/audio_files_drop
Processing file: /Users/lheyerda/Dropbox/STICK-EM/transcriber/audio_files_drop/test_audio_short.wav


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Error processing file /Users/lheyerda/Dropbox/STICK-EM/transcriber/audio_files_drop/test_audio_short.wav: Failed to load audio: ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype

In [ ]:
# Initialize Whisper model
model = whisper.load_model("large-v2")

In [3]:
import urllib.request
import ssl
import whisper

# Disable SSL certificate verification
#ssl._create_default_https_context = ssl._create_unverified_context



# Path to the audio file
file = "/Users/lheyerda/Dropbox/test_audio_short.wav"
print(file,"file")
if ".wav" in file:
    output_audio_file = file
else:
    # Paths to the audio files
    input_audio_file = file
    output_audio_file = file[:-3] + "wav"

    # Convert audio file to WAV format
    audio = AudioSegment.from_file(input_audio_file)
    audio.export(output_audio_file, format="wav")

# Transcribe audio using Whisper
result = model.transcribe(output_audio_file)

# Save transcription to a file
with open("/Users/lheyerda/Downloads/annotated_transcript.txt", "w") as f:
    for segment in result['segments']:
        start = segment['start']
        end = segment['end']
        text = segment['text']
        f.write(f"{start} - {end}: {text}\n")

print("Transcription saved.")

/Users/lheyerda/Dropbox/test_audio_short.wav file


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


RuntimeError: Failed to load audio: ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --ena